In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Simple ADK Agent

> **Note**: This notebook is for reference and educational purposes only. Not intended for production use.  
> **Questions?** mateuswagner@google.com

This notebook demonstrates how to build a simple agent using Google's Agent Development Kit (ADK) inside a jupyter notebook

**Tech Stack**: ADK + Gemini 3


---

## Get started

In [ ]:
## Installation
# Python 3.10+ recommended
# Virtual environment (venv) recommended for isolation
# Google Cloud SDK initialized: `gcloud init`

%pip install --upgrade --quiet 'google-adk' nbformat

# Restart your Jupyter kernel !


## Import libraries

In [1]:
# Standard library imports
import asyncio
import json
import os
import warnings
from typing import Any

# Third-party imports
from IPython.display import Markdown, display

# Google ADK imports
from google.adk.agents import Agent
from google.adk.events import Event
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

### CONFIGURATION SETTINGS

In [2]:
# Google Cloud Project Configuration
# CHANGE THIS: Set your Google Cloud project ID
PROJECT_ID = "matt-demos" # CHANGE IT!

# Fallback to environment variable if not set
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

# Default region for Vertex AI resources
LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "global")

# Set environment variables
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
os.environ["GOOGLE_CLOUD_LOCATION"] = LOCATION
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "True"

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

## Build ADK agent

Build your application using ADK, including the Gemini model and custom tools that you define.

---


### Set agent tools

To start, set the tools that a customer support agent needs to do their job.

In [3]:
def get_product_details(product_name: str):
    """Gathers basic details about a product."""
    details = {
        "smartphone": "A cutting-edge smartphone with advanced camera features and lightning-fast processing.",
        "usb charger": "A super fast and light usb charger",
        "shoes": "High-performance running shoes designed for comfort, support, and speed.",
        "headphones": "Wireless headphones with advanced noise cancellation technology for immersive audio.",
        "speaker": "A voice-controlled smart speaker that plays music, sets alarms, and controls smart home devices.",
    }
    return details.get(product_name, "Product details not found.")


def get_product_price(product_name: str):
    """Gathers price about a product."""
    details = {
        "smartphone": 500,
        "usb charger": 10,
        "shoes": 100,
        "headphones": 50,
        "speaker": 80,
    }
    return details.get(product_name, "Product price not found.")

### Set Agent the model

Configure the Gemini model for your ADK agent. This notebook uses **`gemini-3-pro-preview`** for fast, cost-effective function calling.

See the [Gemini model documentation](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models) for detailed performance benchmarks and pricing.

In [4]:
model = "gemini-3-pro-preview"

### Assemble the ADK Agent

In [5]:
async def agent_parsed_outcome(query):
   # Session identifiers for tracking agent interactions
   app_name = "product_research_app"
   user_id = "user_one"
   session_id = "session_one"
   
   # Create agent with dynamic instruction based on query
   product_research_agent = Agent(
       name="ProductResearchAgent",
       model=model,
       description="An agent that performs product research.",
       instruction=f"""
       Analyze this user request: '{query}'.
       If the request is about price, use get_product_price tool.
       Otherwise, use get_product_details tool to get product information.
       """,
       tools=[get_product_details, get_product_price],
   )

   # Initialize in-memory session storage
   session_service = InMemorySessionService()
   await session_service.create_session(
       app_name=app_name, user_id=user_id, session_id=session_id
       )

   # Create runner to execute agent with session management
   runner = Runner(
       agent=product_research_agent, app_name=app_name, session_service=session_service,
   )

   # Format query as user message and run agent asynchronously
   content = types.Content(role="user", parts=[types.Part(text=query)])
   events = [event async for event in runner.run_async(user_id=user_id, session_id=session_id, new_message=content)]
   
   # Parse events into dictionary with response and tool calls
   return parse_adk_output_to_dictionary(events)

In [6]:
# Agent Wrapper and helpers

def agent_parsed_outcome_sync(prompt: str):
    return asyncio.run(agent_parsed_outcome(prompt))


def parse_adk_output_to_dictionary(events: list[Event]):
    """Parse ADK event output into a structured dictionary format."""

    final_response = ""
    trajectory = []

    for event in events:
        if not getattr(event, "content", None) or not getattr(event.content, "parts", None):
            continue
        for part in event.content.parts:
            if getattr(part, "function_call", None):
                info = {
                    "tool_name": part.function_call.name,
                    "tool_input": dict(part.function_call.args),
                }
                if info not in trajectory:
                    trajectory.append(info)
            if event.content.role == "model" and getattr(part, "text", None):
                final_response = part.text.strip()

    return {"response": final_response, "predicted_trajectory": trajectory}


def format_output_as_markdown(output: dict) -> str:
    """Convert the output dictionary to a formatted markdown string."""
    markdown = "### AI Response\n" + output["response"] + "\n\n"
    if output["predicted_trajectory"]:
        markdown += "### Function Calls\n"
        for call in output["predicted_trajectory"]:
            markdown += f"- **Function**: `{call['tool_name']}`\n"
            markdown += "  - **Arguments**\n"
            for key, value in call["tool_input"].items():
                markdown += f"    - `{key}`: `{value}`\n"
    return markdown

In [7]:
# Test the agent

response = await agent_parsed_outcome(query="Get product details for shoes")
display(Markdown(format_output_as_markdown(response)))

### AI Response
The product details for shoes are: High-performance running shoes designed for comfort, support, and speed.

### Function Calls
- **Function**: `get_product_details`
  - **Arguments**
    - `product_name`: `shoes`


---

# Thank you